In [85]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import pearsonr
import scanpy as sc
from math import log
from statistics import median

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Input
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras import backend as K
import keras.losses

from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
adata = sc.read_10x_mtx(
    '/Volumes/Samsung_T5/ResearchData/scanpyTutorial/data/filtered_gene_bc_matrices/hg19',  # the directory with the `.mtx` file
    var_names='gene_symbols',                      # use gene symbols for the variable names (variables-axis index)
    cache=True)

adata.var_names_make_unique()

In [43]:
data = pd.DataFrame.sparse.from_spmatrix(adata.X)
print('Working on {} cells and {} genes'.format(*data.shape))

Working on 2700 cells and 32738 genes


In [45]:
# Filter out genes that are not expressed in any cells
geneSum = data.sum(axis=0)
x = geneSum.index[geneSum == 0].tolist()
data = data.drop(x, axis = 1)
data.shape

(2700, 16634)

In [103]:
for j in range(2700):
    cellSum = data.sum(axis=1)[j]
    median_j = median(data.iloc[j,:])
    for i in range(16634):
        if data.iloc[j,i] > 0:
            data.iloc[j,i] = log( ( (data.iloc[j,i])/(cellSum) * median_j ) + 1)

TypeError: SparseArray does not support item assignment via setitem

In [113]:
data.loc[j,i] = log(data.iloc[j,i]/data.sum(axis=1)[j] * median_j + 1)

0.0